<h1><center>Laboratorio 8: Aprendizaje Supervisado 🔮</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Pablo Badilla y Ignacio Meza
- Auxiliar: Sebastián Tinoco
- Ayudante: Felipe Arias y Diego Cortez

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Canela Orellana
- Nombre de alumno 2: Isabel Marx


### **Link de repositorio de GitHub:** https://github.com/CanelaOrellana/MDS7202-IsabelMarx-y-CanelaOrellana

## Temas a tratar

- Aprendizaje Supervisado
- Flujos de datos a través de `Pipelines`.

## Reglas

- **Grupos de 2 personas**
- Asistencia **obligatoria** a instrucciones del lab (viernes 16.15). Luego, pueden quedarse trabajando en las salas o irse.
- **No se revisarán entregas de personas ausentes**. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.

## Objetivos principales del laboratorio

- Comprender el funcionamiento de clasificadores/regresores.
- Generar múltiples modelos predictivos.
- Comprender las ventajas de crear modelos en pipeline vs hacer las operaciones a mano.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre `DataFrames`.

### Importamos librerias utiles 😸

In [64]:
# Libreria Core del lab.
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split 

# Pre-procesamiento
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

# Metricas de evaluación
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score

#Libreria para plotear
!pip install --upgrade plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


https://www.kaggle.com/antoinekrajnc/soccer-players-statistics

# 1. Predicciones Futboleras

<p align="center">
  <img src="https://drive.google.com/uc?export=view&id=1BnO4tyh3vM2P199Ec9s3JjngQ4qQ9seP"
" width="300">
</p>


Tras el trágico despido de la mítica mascota de Maipú, Renacín decide adentrarse como consultor en el mercado futbolero, el cuál (para variar...) está cargado en especulaciones. 

Como su principal tarea será asesorar a los directivos de los clubes sobre cuál jugador comprar y cuál no, Renacín desea generar modelos predictivos que evaluén distintas características de los jugadores; todo con el fin de tomar decisiones concretas basadas en los datos.

Sin embargo, su condición de corporeo le impidió tomar la versión anterior de MDS7202, por lo que este motivo Renacín contrata a su equipo para lograr su objetivo final. Dado que aún tiene fuertes vínculos con la dirección de deportes de la municipalidad, el corporeo le entrega base de datos con las estadísticas de cada jugador para que su equipo empieze a trabajar ya con un dataset listo para ser usado.


**Los Datos**

Para este laboratorio deberán trabajar con los csv `statsplayers.csv` y `salarios.pickle`, donde deberán aplicar algoritmos de de aprendizaje supervisado (clasificación y regresión) en base a características que describen de jugadores de futbol.

Para comenzar cargue el dataset señalado y a continuación vea el reporte **`Player_Stats_Report.html`** (adjunto en la carpeta del enunciado) que describe las características principales del `DataFrame`.

In [65]:
# Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
try:
    from google.colab import drive
    drive.mount("/content/drive")
    path = '/content/drive/MyDrive/lab8'
except: 
    print('Ignorando conexión drive-colab')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [66]:
df_players = pd.read_csv(path+'/stats_players.csv')
df_players

,Name,Nationality,National_Position,Club_Position,Height,Weight,Preffered_Foot,Age,Work_Rate,Weak_foot,...,Agility,Jumping,Heading,Shot_Power,Finishing,Long_Shots,Curve,Freekick_Accuracy,Penalties,Volleys
0,Cristiano Ronaldo,Portugal,LS,LW,185,80,Right,32,High / Low,4,...,90,95,85,92,93,90,81,76,85,88
1,Lionel Messi,Argentina,RW,RW,170,72,Left,29,Medium / Medium,4,...,90,68,71,85,95,88,89,90,74,85
2,Neymar,Brazil,LW,LW,174,68,Right,25,High / Medium,5,...,96,61,62,78,89,77,79,84,81,83
3,Luis Suárez,Uruguay,LS,ST,182,85,Right,30,High / Medium,4,...,86,69,77,87,94,86,86,84,85,88
4,Manuel Neuer,Germany,GK,GK,193,92,Right,31,Medium / Medium,4,...,52,78,25,25,13,16,14,11,47,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17583,Adam Dunbar,Republic of Ireland,NaN,Sub,183,82,Right,19,Medium / Medium,1,...,27,56,14,16,13,13,11,13,15,12
17584,Dylan McGoey,Republic of Ireland,NaN,Sub,185,80,Right,19,Medium / Medium,2,...,28,53,12,17,12,11,12,13,16,12
17585,Tommy Ouldridge,England,NaN,Res,173,61,Right,18,High / Medium,2,...,54,61,41,44,28,42,35,36,42,37
17586,Mark Foden,Scotland,NaN,Sub,180,80,Right,21,Medium / Medium,3,...,34,48,15,23,14,12,13,12,24,12


## 1.1 Predicción de Seleccionados Nacionales 

Como primera tarea, Renacín, intrigado por la posibilidad de saber qué tan reconocido es un jugador, le consulta a su equipo si es posible predecir si un jugador será o no seleccionado nacional a partir de sus estadísticas en el juego. 

<p align="center">
  <img src="https://thumbs.gfycat.com/ThornyFrankFoxhound-size_restricted.gif" width="500">
</p>

### 1.1.1 Generación de Labels para la Clasificación [Sin Puntaje]


Primero comience generando las labels para la clasificación. Para esto, trabaje sobre el atributo ```National_Position``` suponiendo que los valores nulos son jugadores no seleccionados para representar a su país. 

Hecho esto, ¿Cuantos ejemplos por cada clase se tienen? Comente lo que observa.



**Respuesta:**

A continuación se crea la nueva columna a partir de ```National_Position```, se revisa si el valor es nulo y se iguala a ```False``` para dejar en ```True``` a aquellos que si serían seleccionados y en ```False``` a los que no.

Luego, se transforman a enteros los valores booleanos definidos, quedando:
- **1:** jugador seleccionado
- **0:** jugador no seleccionado

In [67]:
#Se crea la nueva columna a partir de National Position
df_players['Seleccionado'] = df_players['National_Position'].isna()==False
#Se transforman a enteros
df_players['Seleccionado'] = df_players['Seleccionado'].astype(int)

df_players.head(5)

,Name,Nationality,National_Position,Club_Position,Height,Weight,Preffered_Foot,Age,Work_Rate,Weak_foot,...,Jumping,Heading,Shot_Power,Finishing,Long_Shots,Curve,Freekick_Accuracy,Penalties,Volleys,Seleccionado
0,Cristiano Ronaldo,Portugal,LS,LW,185,80,Right,32,High / Low,4,...,95,85,92,93,90,81,76,85,88,1
1,Lionel Messi,Argentina,RW,RW,170,72,Left,29,Medium / Medium,4,...,68,71,85,95,88,89,90,74,85,1
2,Neymar,Brazil,LW,LW,174,68,Right,25,High / Medium,5,...,61,62,78,89,77,79,84,81,83,1
3,Luis Suárez,Uruguay,LS,ST,182,85,Right,30,High / Medium,4,...,69,77,87,94,86,86,84,85,88,1
4,Manuel Neuer,Germany,GK,GK,193,92,Right,31,Medium / Medium,4,...,78,25,25,13,16,14,11,47,11,1


In [68]:
df_players['Seleccionado'].value_counts()

0    16513
1     1075
Name: Seleccionado, dtype: int64

Se tienen 
- 16.513 jugadores no seleccionados
- 1.075 jugadores seleccionados

Esto era esperable, ya que las selecciones de los países no son tan grandes y por supuesto existen muchos más equipos a los que el resto de los jugadores puede pertenecer.

### 1.1.2 Camino a la clasificación [1 punto]

<p align="center">
  <img src="https://c.tenor.com/SoXZJHWnhKAAAAAC/pitbull-medel.gif" width="500">
</p>

Para preprocesar el dataset, genere un `ColumnTransformer` en donde especifique las transformaciones que hay que realizar para cada columna (por ejemplo StandarScaler, MinMaxScaler, OneHotEncoder, etc...) para que puedan ser utilizadas correctamente por el modelo predictivo y guardelo en algúna variable.

Luego, comente y justifique las transformaciones elegidas sobre cada una de las variables (para esto utilice el material ```Player_Stats_Report.html``` que viene en el zip del lab), al igual que las transformaciones aplicadas.

Hecho lo anterior, **defina al menos 3 pipelines para la clasificación**, en donde utilice el mismo ColumnTransformer definido anteriormente, pero que varie entre cada pipeline los clasificadores. 

Para seleccionar los clasificadores más adecuados, utilice la siguiente guía:

<p align="center">
  <img src="https://scikit-learn.org/stable/_static/ml_map.png" width="500">
</p>

<br/>

Con ella, comente y justifique cada una de las decisiones tomadas al momento de desarrollar su pipeline.

**Nota:** Si tiene problemas al utilizar OneHotEncoder puede utilizar el parámetro handle_unknown='ignore'. Esto hace que en la codificación se omitan las categorias que no aparecen en el entrenamiento. Pregunta dudosa (no tiene puntaje), ¿esto tiene sentido a nivel de modelos?.

**To-Do:**

- [x] Genere un ```ColumnTransformer``` enfocado en preprocesar los datos.
- [x] Indicar y Justificar que preprocesamiento utiliza sobre cada columna.
- [x] Crear 3 ``pipelines`` con diferentes clasificadores.
- [x] Para seleccionar los clasificadores base sus decisiones en la siguiente [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html)
- [x] No entrenar los ``pipelines`` aún.

**Nota:** No es necesario entrenar los clasificadores aún.

In [69]:
ct = ColumnTransformer(
    transformers=[
        ('OneHotEncoder', OneHotEncoder(),  ['Preffered_Foot', 'Work_Rate', 'Weak_foot', 'Skill_Moves']),
        ('StandardScaler', StandardScaler(), ['Height', 'Weight', 'Reactions', 'Balance', 'Agility', 'Jumping']),
        ('MinMax', MinMaxScaler(),['Age', 'Aggression', 'Vision', 'Composure', 'Acceleration', 'Speed', 'Strength','Finishing',
                                   'Long_Shots','Volleys']), 
        ('PowerTransform', PowerTransformer() , ['Ball_Control', 'Dribbling', 'Marking', 'Sliding_Tackle','Standing_Tackle', 
                                                 'Interceptions', 'Crossing', 'Short_Pass', 'Long_Pass', 'Stamina', 'Heading',
                                                 'Shot_Power','Curve', 'Freekick_Accuracy', 'Penalties'])])

Se pre procesaran las columnas del dataframe de la siguiente manera: 

- Aplicación de OneHotEncoding: se aplicó a aquellas columnas categóricas, que tuvieran un número prudente de categorias distintas, estas fueron: 
  - ```'Preffered_Foot'```
  - ```'Work_Rate'```
  - ```'Weak_foot'```
  - ```'Skill_Moves'```  

  No se aplicó ni al nombre, ni al país, ya que son demasiadas categorías y no se considera información relevante al clasificar. Tampoco a ```National_Position``` ni ```Club_Position```, ya que la primera tiene un montón de nulos y se ve expresada en ```Seleccionado``` y la segunda tiene demasiadas categorías posibles distintas.  

- Aplicación de StandarScaler: Se aplicó esta transformación a las columnas que se observó en el reporte que seguían a una distribución similar a la normal, estas fueron: 
  - ```'Height'``` 
  - ```'Weight'``` 
  - ```'Reactions'```
  - ```'Balance'```
  - ```'Agility'```
  - ```'Jumping'```

- Aplicación de MinMaxScaler: Se aplicó a aquellas columnas numéricas, cuya distribución no fuera tan similar a una normal, pero tampoco tan distante, estas fueron:
  - ```'Age'```
  - ```'Aggression'```
  - ```'Vision'```
  - ```'Composure'```
  - ```'Acceleration'```
  - ```'Speed'```
  - ```'Strength'```
  - ```'Finishing'```
  - ```'Long_Shots'```
  - ```'Volleys'```

- Aplicación de PowerTransform: Se aplicó a aquellas columnas con una distribución muy diferente a la normal, las escogidas fueron:
  - ```'Ball_Control'```
  - ```'Dribbling'```
  - ```'Marking'```
  - ```'Sliding_Tackle'```
  - ```'Standing_Tackle'```
  - ```'Interceptions'```
  - ```'Crossing'```
  - ```'Short_Pass'```
  - ```'Long_Pass'```
  - ```'Stamina'```
  - ```'Heading'```
  - ```'Shot_Power'```
  - ```'Curve'```
  - ```'Freekick_Accuracy'```
  - ```'Penalties'```

**Respuesta:**

Siguiendo la guía entregada, se realizóel siguiente camino:
- más de 50 muestras
- prediciendo categorías
- data etiquetada
- menos de 100K muestras, así quedaron las siguientes opciones en orden:
  - Linear SCV
  - KNeighbors Classifier
  - SCV
  - Ensemble Classifiers  

Por lo que se utilizaron los primeros 3 para realizar los pipelines.

In [70]:
#Se crean los clasificadores
L_svc = LinearSVC()
knn = KNeighborsClassifier()
svc = SVC()

#Se crea un pipeline para cada transformador
pipe_L_svc = Pipeline([("ColumnTransformer", ct),("LinearSVC", L_svc)])
pipe_knn = Pipeline([("ColumnTransformer", ct),("KNeighbors", knn)])
pipe_svc = Pipeline([("ColumnTransformer", ct),("SVC", svc)])

### 1.1.3 Entrenemos los pipelines [1 punto]

<p align="center">
  <img src="https://i.gifer.com/7Cc8.gif" width="350">
</p>


Ahora, entrene los `pipeline` generados en los pasos anteriores. Para esto, primero separe los datos de entrenamiento en un conjunto de entrenamiento y de prueba  (la proporción queda a su juicio).

En este paso, seleccione los ejemplos de forma aleatoria e intente mantener la distribución original de labels de cada clase en los conjuntos de prueba/entrenamiento. (vea la documentación de `train_test_split`).


Luego, entrene los pipelines

Una vez entrenado su modelo, evalue su rendimiento a través de diferentes métricas, comentando que significa cada uno de los valores obtenidos. Puede usar la función `classification_report` para corroborar sus resultados.

- ¿Qué implican los valores de accuracy, precisión y recall de la clase positiva (la que indica que un jugador es seleccionado nacional)?
- ¿Podrían mejorarse los resultados?, ¿Cómo?
- ¿Influye la cantidad de ejemplos por clase?

**To-Do**:

- [x] Separar el conjunto de datos en entrenamiento y de prueba.
- [x] Entrenar los `pipelines`.
- [x] Utilizar las métricas para evaluar los modelos generados.
- [ ] Responder preguntas

**Respuesta:**

In [71]:
from sklearn.model_selection import train_test_split
y = df_players['Seleccionado']
X = df_players.drop(columns=['Seleccionado'])


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [72]:
y_train.value_counts(), y_test.value_counts()

(0    11052
 1      731
 Name: Seleccionado, dtype: int64,
 0    5461
 1     344
 Name: Seleccionado, dtype: int64)

In [73]:
11052/731, 5461/344

(15.119015047879618, 15.875)

Para ambos conjuntos la proporción de no_seleccionados/seleccionados se encuentra entre 15 y 16, por lo que nos parece que las proporciones se mantienen de manera correcta.

In [74]:
#Se entrenan los 3 clasificadores
pipe_L_svc.fit(X_train, y_train)
pipe_knn.fit(X_train, y_train)
pipe_svc.fit(X_train, y_train)

#Se predice con los 3 clasificadores
y_Lsvc = pipe_L_svc.predict(X_test)
y_knn = pipe_knn.predict(X_test)
y_svc = pipe_svc.predict(X_test)


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [75]:
print('Clasificación por Linear SVC')
print("Matriz de confusión\n", confusion_matrix(y_Lsvc, y_test))
print(classification_report(y_test, y_Lsvc))

Clasificación por Linear SVC
Matriz de confusión
 [[5461  343]
 [   0    1]]
              precision    recall  f1-score   support

           0       0.94      1.00      0.97      5461
           1       1.00      0.00      0.01       344

    accuracy                           0.94      5805
   macro avg       0.97      0.50      0.49      5805
weighted avg       0.94      0.94      0.91      5805



In [76]:
print('Clasificación por KNeighbors Classifier')
print("Matriz de confusión\n", confusion_matrix(y_knn, y_test))
print(classification_report(y_test, y_knn))

Clasificación por KNeighbors Classifier
Matriz de confusión
 [[5430  313]
 [  31   31]]
              precision    recall  f1-score   support

           0       0.95      0.99      0.97      5461
           1       0.50      0.09      0.15       344

    accuracy                           0.94      5805
   macro avg       0.72      0.54      0.56      5805
weighted avg       0.92      0.94      0.92      5805



In [77]:
print('Clasificación por SVC')
print("Matriz de confusión\n", confusion_matrix(y_svc, y_test))
print(classification_report(y_test, y_svc))

Clasificación por SVC
Matriz de confusión
 [[5461  343]
 [   0    1]]
              precision    recall  f1-score   support

           0       0.94      1.00      0.97      5461
           1       1.00      0.00      0.01       344

    accuracy                           0.94      5805
   macro avg       0.97      0.50      0.49      5805
weighted avg       0.94      0.94      0.91      5805



**Respuesta**:

¿Qué implican los valores de accuracy, precisión y recall de la clase positiva (la que indica que un jugador es seleccionado nacional)?  
¿Podrían mejorarse los resultados?, ¿Cómo?  
¿Influye la cantidad de ejemplos por clase?



Dados los rendimientos en precision, recall y f1-score, además por supuesto de lo arrojado por la matriz de confusión, se da cuenta de que los clasificadores están prediciendo todos (SVC y linear SVC) o la mayoría (KNeighbors) de los valores como 0, por lo que no logran reconocer a los futbolistas seleccionados.  

Esto se debe al gran desbalance de las clases que se desean clasificar, ya que los clasificadores como SVC consiguen disminuir las veces que se equivocan si dicen que todos los valores son 0. Mientras que KNeighbors no tiene suficientes ejemplos para aprender quienes son los seleccionados.

## 1.2 Predicción de posiciones de jugadores [2 puntos]

<p align="center">
  <img src="https://pbs.twimg.com/media/E1rfA1aWEAYU6Ny.jpg" width="500">
</p>

En una nueva jornada de desmesuradas transacciones deportivas, Renacín escuchó a sus colegas discutir acerca de que el precio de cada jugador depende en gran medida de la posición en la cancha en la que juega. Y además, que hay bastantes jugadores nuevos que no tienen muy claro en que posición verdaderamente brillarían, por lo que actualmente puede que actualmente estén jugando en posiciones sub-optimas. 

Viendo que los resultados del primer análisis no son tan esperanzadores, el corporeo los comanda a cambiar su tarea: ahora, les solicita que construyan un clasificador enfocado en predecir la mejor posición de los jugadores en la cancha según sus características. 

Para lograr esto, primero, les pide que etiqueten de la siguiente manera los valores que aparecen en el atributo ```Club_Position```, pidiendo que agrupen los valores en los siguientes grupos:

**Nota**:  Renacín les recalca que **no deben utilizar los valores ```Sub``` y ```Res``` de esta columna**.

```python
ataque = ['ST', 'CF'] 
central_ataque = ['RW', 'CAM', 'LW'] 
central = ['RM', 'CM', 'LM'] 
central_defensa = ['RWB', 'CDM', 'LWB']
defensa = ['RB', 'CB', 'LB']
arquero = ['GK']
```

Cabe señalar que al igual como lo realizado con la clasificación binaria, deberá justificar en base a la [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html) la elección del clasificador y se deben comentar los resultados obtenidos en la clasificación.

**Nota:** Clasifique solamente con las clases señaladas, si observa mas clases eliminelas de la clasficación.

**To-Do**:

- [x] Aplique las etiquetas descritas anteriormente en cada uno de los valores señalados en esta sección.
- [x] Cuente cuantos por clase quedan.
- [x] Entrene el nuevo pipeline y ejecute una evaluación de este.  
- [ ] Comente los resultados obtenidos.

**Respuesta:**

In [78]:
#Aplicar etiquetas
posiciones = {'ST':'ataque',
              'CF':'ataque',
              'RW':'central_ataque',
              'CAM':'central_ataque',
              'LW':'central_ataque',
              'RM':'central',
              'CM':'central',
              'LM':'central',
              'RWB':'central_defensa',
              'CDM':'central_defensa',
              'LWB':'central_defensa',
              'RB':'defensa',
              'CB':'defensa',
              'LB':'defensa',
              'GK':'arquero'}

df_players['Club_Position'] = df_players['Club_Position'].map(posiciones)

df_players.dropna(subset=['Club_Position'], inplace=True)

In [80]:
#Cuenta de cuántos quedan por clase
df_players['Club_Position'].value_counts()

defensa            1180
central             907
arquero             632
central_ataque      581
ataque              430
central_defensa     209
Name: Club_Position, dtype: int64

Observando la guía, se decide el clasificador multiclases a utilizar de la siguiente manera:

- más de 50 muestras
- prediciendo categorías
- data etiquetada
- menos de 100K muestras, así quedaron las siguientes opciones en orden:
  - Linear SCV
  - KNeighbors Classifier
  - SCV
  - Ensemble Classifiers  

Dado que el camino es el mismo que el anterior, se decide utilizar la opción que anteriormente se dejó fuera *Ensamble Classifiers*, en particular la opción de Random Forest, ya que es la que suena más conocida.



In [81]:
from sklearn.ensemble import RandomForestClassifier

#Se crea el clasificador
rfc = RandomForestClassifier()

#Se crea un pipeline para cada transformador
pipe_rf = Pipeline([("ColumnTransformer", ct),("RandomForest", rfc)])

In [82]:
y = df_players['Club_Position']
X = df_players.drop(columns=['Club_Position'])


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [83]:
#Se entrena el clasificador
pipe_rf.fit(X_train, y_train)

#Se predice con los 3 clasificadores
y_rf = pipe_rf.predict(X_test)

In [84]:
print('Clasificación por Random Forest')
print("Matriz de confusión\n", confusion_matrix(y_rf, y_test))
print(classification_report(y_test, y_rf))

Clasificación por Random Forest
Matriz de confusión
 [[200   0   0   0   0   0]
 [  0 124  15  11   0   0]
 [  0  16 197  93   8   4]
 [  0   5  57  72   5   2]
 [  0   0   5   2  15   6]
 [  0   0  42   5  40 376]]
                 precision    recall  f1-score   support

        arquero       1.00      1.00      1.00       200
         ataque       0.83      0.86      0.84       145
        central       0.62      0.62      0.62       316
 central_ataque       0.51      0.39      0.44       183
central_defensa       0.54      0.22      0.31        68
        defensa       0.81      0.97      0.88       388

       accuracy                           0.76      1300
      macro avg       0.72      0.68      0.68      1300
   weighted avg       0.74      0.76      0.74      1300



**Respuesta**:

```
Comente de resultados observados y si es posible mejorar los resultados.
```

## 1.3 Predicción de Sueldos [2 puntos]

<p align="center">
  <img src="https://media.giphy.com/media/3oEjHWtHAFcOKS4iA0/giphy.gif" width="460">
</p>

Queriendo ahondar aún más en el mercado del balompíe, Renacin, logra obtener (de una manera no muy formal) los sueldos de múltiples futbolistas y los guarda en el archivo ```sueldos.csv```. Con ellos les solicita que generen un regresor que les permita predecir el sueldo de los futbolistas en base a las características de los pichichis, esto, debido a su motivación por invertir y/o realizar especulación sobre los sueldos de jugadores.

Renacin es claro señalando que deben seguir utilizando la [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html) y comenten cada uno de los pasos realizados, para obtener su regresión lineal. Señalándoles que no aceptara un $R^2$ inferior a 0.35 para el modelo solicitado.

Para esta parte usted tiene total libertad en la generación del regresor, la unica exigencia es que utilice un pipeline para generar la regresión y utilice la metrica $R^2$ para medir el rendimiento de esta.

**To-Do**:

- [x] Explique en que consiste la métrica $R^2$
- [ ] Generar un pipeline para la regresión.
- [ ] Obtener un regresor con un $R^2$ superior a  $0.35$.
- [ ] Comente sus resultados y si es posible mejorar los resultados obtenidos. ¿Se necesitarían más datos o otros tipos de características o una combinación de ambos?

**Respuesta**

La métrica $R^{2}$ nos dice qué porcentaje de la variabilidad de los datos que se están prediciendo, se logra explicar con el modelo propuesto para la regresión.

Para elegir la regresión a utilizar, se mira la guía, se sigue el siguiente camino:

- más de 50 muestras
- prediciendo cantidades
- menos de 100K muestras, así quedaron las siguientes opciones en orden:
  - RidgeRegressions
  - SVR
  - SVR(kernel='rbf')
  - Ensemble Regressors
  

In [ ]:
### Código aquí ###

**Respuesta**:

```
Comente de resultados observados y si es posible mejorar los resultados.
```

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media2.giphy.com/media/l1J3CbFgn5o7DGRuE/giphy.gif">
</p>

<br>
<center>
<img src="https://i.kym-cdn.com/photos/images/original/001/194/195/b18.png" width=100 height=50 />
</center>
<br>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>